In [1]:
import pandas as pd
import datetime

In [2]:
store_geomap=pd.read_excel("/home/jian/Projects/Savers/TA_creation/TA_output_Final_Version/Savers Cluster store details_20190227 Tom.xlsx",sheetname="US by store",dtype=str)
store_geomap=store_geomap[['LOCATIONNUMBER','GEOMAP']].drop_duplicates().rename(columns={"LOCATIONNUMBER":"location_id","GEOMAP":"Store_GEOMAP"})

In [3]:
CA_TA_Excel=pd.ExcelFile("/home/jian/Projects/Savers/TA_creation/TA_output_Final_Version/Savers_CA_TA_Final_Version_JL_2019-03-22.xlsx")
CA_TA_Excel.sheet_names

['Store_list', 'TA_Summary', 'TA_Summary_by_store_count', 'TA_zips']

In [4]:
CA_store_list=CA_TA_Excel.parse('Store_list',dtype=str)
CA_store_list=CA_store_list[['location_id','City','St/Province']].rename(columns={"City":"city_of_store","St/Province":"Province_of_store"})
CA_TA_zip=CA_TA_Excel.parse('TA_zips',dtype=str)
print(CA_TA_zip.shape)
CA_TA_zip=pd.merge(CA_TA_zip,CA_store_list,on="location_id",how="left")
CA_TA_zip=pd.merge(CA_TA_zip,store_geomap,on="location_id",how="left")
print(CA_TA_zip.shape)

(981422, 5)
(981422, 8)


In [5]:
CA_zip_state=pd.read_table("/home/jian/Docs/Geo_mapping/Canadian_Postal_Codes/canada-postal-code-database-BUSINESS-CSV/canada-postal-code-database-BUSINESS.csv",
                          dtype=str,usecols=['PostalCode','Province'],sep=",")
CA_zip_state=CA_zip_state.drop_duplicates()
print(CA_zip_state.shape)
print(len(CA_zip_state['PostalCode'].unique()))

CA_zip_state=CA_zip_state.rename(columns={"PostalCode":"zip_cd","Province":"Province_of_zip"})
CA_zip_state=CA_zip_state.append(pd.DataFrame({"zip_cd":"N3S 7P2","Province_of_zip":"ON"},index=[1]))
CA_zip_state=CA_zip_state.append(pd.DataFrame({"zip_cd":"T1Y 6J6","Province_of_zip":"AB"},index=[1]))
CA_zip_state=CA_zip_state.append(pd.DataFrame({"zip_cd":"L0R 2H2","Province_of_zip":"ON"},index=[1]))

CA_zip_state=CA_zip_state.reset_index()
del CA_zip_state['index']

CA_TA_zip=pd.merge(CA_TA_zip,CA_zip_state,on="zip_cd",how="left")

(873339, 2)
873339


In [6]:
US_TA_Excel=pd.ExcelFile("/home/jian/Projects/Savers/TA_creation/TA_output_Final_Version/Savers_US_TA_Final_Version_JL_2019-03-21.xlsx")
US_TA_Excel.sheet_names

['Store_list',
 'TA_Summary',
 'TA_Summary_by_store_count',
 'TA_zips',
 'tableau_data',
 'maps_1_store_TA_5_miles_plus',
 'maps_2_plus_stores_TA']

In [7]:
US_store_list=US_TA_Excel.parse('Store_list',dtype=str)
US_store_list=US_store_list[['location_id','City','St/Province']].rename(columns={"City":"city_of_store","St/Province":"St_of_store"})
US_TA_zip=US_TA_Excel.parse('TA_zips',dtype=str)
print(US_TA_zip.shape)
US_TA_zip=pd.merge(US_TA_zip,US_store_list,on="location_id",how="left")
US_TA_zip=pd.merge(US_TA_zip,store_geomap,on="location_id",how="left")
print(US_TA_zip.shape)

(1850, 5)
(1850, 8)


In [8]:
US_zip_state_from_Nielsen=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str)
US_zip_state_from_Nielsen=US_zip_state_from_Nielsen.rename(columns={"ZIP":"zip_cd","ST":"st_of_zip"})
US_zip_state_from_Nielsen=US_zip_state_from_Nielsen.loc[1:,['zip_cd','st_of_zip']]
US_zip_state_from_Nielsen=US_zip_state_from_Nielsen.drop_duplicates().reset_index()
del US_zip_state_from_Nielsen['index']

In [9]:
US_TA_zip=pd.merge(US_TA_zip,US_zip_state_from_Nielsen,on="zip_cd",how="left")
print(US_TA_zip.shape)

(1850, 9)


In [10]:
writer_TA_zip=pd.ExcelWriter("/home/jian/Projects/Savers/TA_creation/TA_output_Final_Version/Excel_of_TA_zip_with_City_State_GEOMAP_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
US_TA_zip.to_excel(writer_TA_zip,"US_ta_zips",index=False)
CA_TA_zip.to_excel(writer_TA_zip,"CA_ta_zips",index=False)
writer_TA_zip.save()